# Prediction of the performance of students in a particular student based on the subject code(taking last 2 digits into considration) ,Output = TOTAL MARKS .....Using ML libraries

### importing required modules

In [1]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder

### creating Spark session

In [2]:
spark = SparkSession.builder.appName('Practise').getOrCreate()

22/04/10 19:28:54 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.195 instead (on interface wlp0s20f3)
22/04/10 19:28:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/10 19:28:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/10 19:28:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/10 19:28:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### reading csv file

In [3]:
df = spark.read.csv('Results_Data.csv',header=True,inferSchema=True)
df.show()

+----------+--------+---+---+---+------+-----+-----+--------+
|       USN|   SCODE|CIE|SEE|TOT|RESULT|GRADE| DATE|ExamType|
+----------+--------+---+---+---+------+-----+-----+--------+
|4JN18CS001| 18MAT11| 32| 41| 73|     P|    8|12019| Regular|
|4JN18CS001| 18PHY12| 29| 47| 76|     P|    8|12019| Regular|
|4JN18CS001| 18ELE13| 25| 43| 68|     P|    7|12019| Regular|
|4JN18CS001| 18CIV14| 31| 26| 57|     P|    6|12019| Regular|
|4JN18CS001|18EGDL15| 31| 56| 87|     P|    9|12019| Regular|
|4JN18CS001|18PHYL16| 32| 40| 72|     P|    8|12019| Regular|
|4JN18CS001|18ELEL17| 34| 52| 86|     P|    9|12019| Regular|
|4JN18CS001| 18EGH18| 27| 32| 59|     P|    6|12019| Regular|
|4JN18CS002| 18MAT11| 33| 51| 84|     P|    9|12019| Regular|
|4JN18CS002| 18PHY12| 36| 45| 81|     P|    9|12019| Regular|
|4JN18CS002| 18ELE13| 37| 31| 68|     P|    7|12019| Regular|
|4JN18CS002| 18CIV14| 34| 29| 63|     P|    7|12019| Regular|
|4JN18CS002|18EGDL15| 34| 55| 89|     P|    9|12019| Regular|
|4JN18CS

### preprosessnig dataframe

In [4]:
df.createOrReplaceTempView("df2_view")

dffinal = spark.sql(
    """
    SELECT USN,TOT,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN DATE LIKE '%19' THEN 19
    WHEN DATE LIKE '%20' THEN 20
    END) AS NEWDATE
    
    FROM df2_view
    """
)

dffinal.show()

+----------+---+----------+-------+
|       USN|TOT|SCODEFINAL|NEWDATE|
+----------+---+----------+-------+
|4JN18CS001| 73|         1|     19|
|4JN18CS001| 76|         2|     19|
|4JN18CS001| 68|         3|     19|
|4JN18CS001| 57|         4|     19|
|4JN18CS001| 87|         5|     19|
|4JN18CS001| 72|         6|     19|
|4JN18CS001| 86|         7|     19|
|4JN18CS001| 59|         8|     19|
|4JN18CS002| 84|         1|     19|
|4JN18CS002| 81|         2|     19|
|4JN18CS002| 68|         3|     19|
|4JN18CS002| 63|         4|     19|
|4JN18CS002| 89|         5|     19|
|4JN18CS002| 75|         6|     19|
|4JN18CS002| 84|         7|     19|
|4JN18CS002| 72|         8|     19|
|4JN18CS003| 81|         1|     19|
|4JN18CS003| 73|         2|     19|
|4JN18CS003| 81|         3|     19|
|4JN18CS003| 72|         4|     19|
+----------+---+----------+-------+
only showing top 20 rows



In [5]:
qualification_indexer = StringIndexer(inputCol="USN", outputCol="EncodedUSN")
#Fits a model to the input dataset with optional parameters.
df1 = qualification_indexer.fit(dffinal).transform(dffinal)
df1.show()

+----------+---+----------+-------+----------+
|       USN|TOT|SCODEFINAL|NEWDATE|EncodedUSN|
+----------+---+----------+-------+----------+
|4JN18CS001| 73|         1|     19|     242.0|
|4JN18CS001| 76|         2|     19|     242.0|
|4JN18CS001| 68|         3|     19|     242.0|
|4JN18CS001| 57|         4|     19|     242.0|
|4JN18CS001| 87|         5|     19|     242.0|
|4JN18CS001| 72|         6|     19|     242.0|
|4JN18CS001| 86|         7|     19|     242.0|
|4JN18CS001| 59|         8|     19|     242.0|
|4JN18CS002| 84|         1|     19|     243.0|
|4JN18CS002| 81|         2|     19|     243.0|
|4JN18CS002| 68|         3|     19|     243.0|
|4JN18CS002| 63|         4|     19|     243.0|
|4JN18CS002| 89|         5|     19|     243.0|
|4JN18CS002| 75|         6|     19|     243.0|
|4JN18CS002| 84|         7|     19|     243.0|
|4JN18CS002| 72|         8|     19|     243.0|
|4JN18CS003| 81|         1|     19|     244.0|
|4JN18CS003| 73|         2|     19|     244.0|
|4JN18CS003| 

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['EncodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output=featureassembler.transform(df1)
output.show()

+----------+---+----------+-------+----------+----------------+
|       USN|TOT|SCODEFINAL|NEWDATE|EncodedUSN|             NEW|
+----------+---+----------+-------+----------+----------------+
|4JN18CS001| 73|         1|     19|     242.0|[242.0,19.0,1.0]|
|4JN18CS001| 76|         2|     19|     242.0|[242.0,19.0,2.0]|
|4JN18CS001| 68|         3|     19|     242.0|[242.0,19.0,3.0]|
|4JN18CS001| 57|         4|     19|     242.0|[242.0,19.0,4.0]|
|4JN18CS001| 87|         5|     19|     242.0|[242.0,19.0,5.0]|
|4JN18CS001| 72|         6|     19|     242.0|[242.0,19.0,6.0]|
|4JN18CS001| 86|         7|     19|     242.0|[242.0,19.0,7.0]|
|4JN18CS001| 59|         8|     19|     242.0|[242.0,19.0,8.0]|
|4JN18CS002| 84|         1|     19|     243.0|[243.0,19.0,1.0]|
|4JN18CS002| 81|         2|     19|     243.0|[243.0,19.0,2.0]|
|4JN18CS002| 68|         3|     19|     243.0|[243.0,19.0,3.0]|
|4JN18CS002| 63|         4|     19|     243.0|[243.0,19.0,4.0]|
|4JN18CS002| 89|         5|     19|     

In [7]:
df2 = output.limit(10000)
df2.show()

+----------+---+----------+-------+----------+----------------+
|       USN|TOT|SCODEFINAL|NEWDATE|EncodedUSN|             NEW|
+----------+---+----------+-------+----------+----------------+
|4JN18CS001| 73|         1|     19|     242.0|[242.0,19.0,1.0]|
|4JN18CS001| 76|         2|     19|     242.0|[242.0,19.0,2.0]|
|4JN18CS001| 68|         3|     19|     242.0|[242.0,19.0,3.0]|
|4JN18CS001| 57|         4|     19|     242.0|[242.0,19.0,4.0]|
|4JN18CS001| 87|         5|     19|     242.0|[242.0,19.0,5.0]|
|4JN18CS001| 72|         6|     19|     242.0|[242.0,19.0,6.0]|
|4JN18CS001| 86|         7|     19|     242.0|[242.0,19.0,7.0]|
|4JN18CS001| 59|         8|     19|     242.0|[242.0,19.0,8.0]|
|4JN18CS002| 84|         1|     19|     243.0|[243.0,19.0,1.0]|
|4JN18CS002| 81|         2|     19|     243.0|[243.0,19.0,2.0]|
|4JN18CS002| 68|         3|     19|     243.0|[243.0,19.0,3.0]|
|4JN18CS002| 63|         4|     19|     243.0|[243.0,19.0,4.0]|
|4JN18CS002| 89|         5|     19|     

In [8]:
finalized_data = df2.select(['NEW','TOT'])
finalized_data.show()

+----------------+---+
|             NEW|TOT|
+----------------+---+
|[242.0,19.0,1.0]| 73|
|[242.0,19.0,2.0]| 76|
|[242.0,19.0,3.0]| 68|
|[242.0,19.0,4.0]| 57|
|[242.0,19.0,5.0]| 87|
|[242.0,19.0,6.0]| 72|
|[242.0,19.0,7.0]| 86|
|[242.0,19.0,8.0]| 59|
|[243.0,19.0,1.0]| 84|
|[243.0,19.0,2.0]| 81|
|[243.0,19.0,3.0]| 68|
|[243.0,19.0,4.0]| 63|
|[243.0,19.0,5.0]| 89|
|[243.0,19.0,6.0]| 75|
|[243.0,19.0,7.0]| 84|
|[243.0,19.0,8.0]| 72|
|[244.0,19.0,1.0]| 81|
|[244.0,19.0,2.0]| 73|
|[244.0,19.0,3.0]| 81|
|[244.0,19.0,4.0]| 72|
+----------------+---+
only showing top 20 rows



In [9]:
finalized_data.printSchema

<bound method DataFrame.printSchema of DataFrame[NEW: vector, TOT: int]>

In [10]:
train, test = finalized_data.randomSplit([0.8, 0.2])
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 8000
Test Dataset Count: 2000


### ML training starts

In [11]:
regressor=LinearRegression(featuresCol='NEW', labelCol='TOT')
regressor=regressor.fit(train)

22/04/10 19:29:02 WARN Instrumentation: [f7075be3] regParam is zero, which might cause numerical instability and overfitting.
22/04/10 19:29:02 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/10 19:29:02 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/10 19:29:02 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
predictions = regressor.transform(test)
predictions.select(['NEW','TOT','prediction']).show(25)

+--------------+---+------------------+
|           NEW|TOT|        prediction|
+--------------+---+------------------+
|[0.0,19.0,1.0]| 30|47.609813006683126|
|[0.0,19.0,1.0]| 34|47.609813006683126|
|[0.0,19.0,3.0]| 37|52.624216805377245|
|[0.0,19.0,4.0]| 48| 55.13141870472431|
|[0.0,19.0,6.0]| 30|60.145822503418415|
|[0.0,19.0,8.0]| 54| 65.16022630211253|
|[1.0,19.0,1.0]| 61| 47.65585988715783|
|[1.0,19.0,2.0]| 51|  50.1630617865049|
|[1.0,19.0,4.0]| 31|55.177465585199016|
|[1.0,19.0,5.0]| 35| 57.68466748454607|
|[1.0,19.0,5.0]| 48| 57.68466748454607|
|[1.0,19.0,6.0]| 55| 60.19186938389312|
|[1.0,19.0,7.0]| 79| 62.69907128324019|
|[1.0,19.0,8.0]| 56| 65.20627318258724|
|[1.0,19.0,8.0]| 58| 65.20627318258724|
|[2.0,19.0,3.0]| 51| 52.71631056632667|
|[2.0,19.0,4.0]| 54|55.223512465673736|
|[2.0,19.0,6.0]| 47| 60.23791626436784|
|[2.0,19.0,8.0]| 56| 65.25232006306196|
|[2.0,19.0,9.0]| 19| 67.75952196240902|
|[2.0,19.0,9.0]| 33| 67.75952196240902|
|[3.0,19.0,1.0]| 35|47.747953648107256|


In [13]:
dftest = spark.read.csv('new1.csv',header=True,inferSchema=True)
dftest.show()
dftest = dftest.limit(15000)

+----------+--------+---+----+
|       USN|   SCODE|TOT|2021|
+----------+--------+---+----+
|4JN18CS001| 18MAT11| 73|2021|
|4JN18CS001| 18PHY12| 76|2021|
|4JN18CS001| 18ELE13| 68|2021|
|4JN18CS001| 18CIV14| 57|2021|
|4JN18CS001|18EGDL15| 87|2021|
|4JN18CS001|18PHYL16| 72|2021|
|4JN18CS001|18ELEL17| 86|2021|
|4JN18CS001| 18EGH18| 59|2021|
|4JN18CS002| 18MAT11| 84|2021|
|4JN18CS002| 18PHY12| 81|2021|
|4JN18CS002| 18ELE13| 68|2021|
|4JN18CS002| 18CIV14| 63|2021|
|4JN18CS002|18EGDL15| 89|2021|
|4JN18CS002|18PHYL16| 75|2021|
|4JN18CS002|18ELEL17| 84|2021|
|4JN18CS002| 18EGH18| 72|2021|
|4JN18CS003| 18MAT11| 81|2021|
|4JN18CS003| 18PHY12| 73|2021|
|4JN18CS003| 18ELE13| 81|2021|
|4JN18CS003| 18CIV14| 72|2021|
+----------+--------+---+----+
only showing top 20 rows



In [14]:
qualification_indexer = StringIndexer(inputCol="USN", outputCol="EncodedUSN")
#Fits a model to the input dataset with optional parameters.
dftest1 = qualification_indexer.fit(dftest).transform(dftest)
dftest1.show()

+----------+--------+---+----+----------+
|       USN|   SCODE|TOT|2021|EncodedUSN|
+----------+--------+---+----+----------+
|4JN18CS001| 18MAT11| 73|2021|     242.0|
|4JN18CS001| 18PHY12| 76|2021|     242.0|
|4JN18CS001| 18ELE13| 68|2021|     242.0|
|4JN18CS001| 18CIV14| 57|2021|     242.0|
|4JN18CS001|18EGDL15| 87|2021|     242.0|
|4JN18CS001|18PHYL16| 72|2021|     242.0|
|4JN18CS001|18ELEL17| 86|2021|     242.0|
|4JN18CS001| 18EGH18| 59|2021|     242.0|
|4JN18CS002| 18MAT11| 84|2021|     243.0|
|4JN18CS002| 18PHY12| 81|2021|     243.0|
|4JN18CS002| 18ELE13| 68|2021|     243.0|
|4JN18CS002| 18CIV14| 63|2021|     243.0|
|4JN18CS002|18EGDL15| 89|2021|     243.0|
|4JN18CS002|18PHYL16| 75|2021|     243.0|
|4JN18CS002|18ELEL17| 84|2021|     243.0|
|4JN18CS002| 18EGH18| 72|2021|     243.0|
|4JN18CS003| 18MAT11| 81|2021|     244.0|
|4JN18CS003| 18PHY12| 73|2021|     244.0|
|4JN18CS003| 18ELE13| 81|2021|     244.0|
|4JN18CS003| 18CIV14| 72|2021|     244.0|
+----------+--------+---+----+----

In [15]:
dftest1.createOrReplaceTempView("df3_view")

dffinal2 = spark.sql(
    """
    SELECT EncodedUSN,TOT,
    (CASE
    WHEN SCODE LIKE '%1' THEN 1
    WHEN SCODE LIKE '%2' THEN 2
    WHEN SCODE LIKE '%3' THEN 3
    WHEN SCODE LIKE '%4' THEN 4
    WHEN SCODE LIKE '%5' THEN 5
    WHEN SCODE LIKE '%6' THEN 6
    WHEN SCODE LIKE '%7' THEN 7
    WHEN SCODE LIKE '%8' THEN 8
    WHEN SCODE LIKE '%9' THEN 9
    END) AS SCODEFINAL,
    (CASE
    WHEN 2021 LIKE '%21' THEN 21
    END) AS NEWDATE
    
    FROM df3_view
    """
)

dffinal2.show()

+----------+---+----------+-------+
|EncodedUSN|TOT|SCODEFINAL|NEWDATE|
+----------+---+----------+-------+
|     242.0| 73|         1|     21|
|     242.0| 76|         2|     21|
|     242.0| 68|         3|     21|
|     242.0| 57|         4|     21|
|     242.0| 87|         5|     21|
|     242.0| 72|         6|     21|
|     242.0| 86|         7|     21|
|     242.0| 59|         8|     21|
|     243.0| 84|         1|     21|
|     243.0| 81|         2|     21|
|     243.0| 68|         3|     21|
|     243.0| 63|         4|     21|
|     243.0| 89|         5|     21|
|     243.0| 75|         6|     21|
|     243.0| 84|         7|     21|
|     243.0| 72|         8|     21|
|     244.0| 81|         1|     21|
|     244.0| 73|         2|     21|
|     244.0| 81|         3|     21|
|     244.0| 72|         4|     21|
+----------+---+----------+-------+
only showing top 20 rows



In [16]:
featureassembler=VectorAssembler(inputCols=['EncodedUSN','NEWDATE','SCODEFINAL'],outputCol="NEW")
output1=featureassembler.transform(dffinal2)
output1.show()

+----------+---+----------+-------+----------------+
|EncodedUSN|TOT|SCODEFINAL|NEWDATE|             NEW|
+----------+---+----------+-------+----------------+
|     242.0| 73|         1|     21|[242.0,21.0,1.0]|
|     242.0| 76|         2|     21|[242.0,21.0,2.0]|
|     242.0| 68|         3|     21|[242.0,21.0,3.0]|
|     242.0| 57|         4|     21|[242.0,21.0,4.0]|
|     242.0| 87|         5|     21|[242.0,21.0,5.0]|
|     242.0| 72|         6|     21|[242.0,21.0,6.0]|
|     242.0| 86|         7|     21|[242.0,21.0,7.0]|
|     242.0| 59|         8|     21|[242.0,21.0,8.0]|
|     243.0| 84|         1|     21|[243.0,21.0,1.0]|
|     243.0| 81|         2|     21|[243.0,21.0,2.0]|
|     243.0| 68|         3|     21|[243.0,21.0,3.0]|
|     243.0| 63|         4|     21|[243.0,21.0,4.0]|
|     243.0| 89|         5|     21|[243.0,21.0,5.0]|
|     243.0| 75|         6|     21|[243.0,21.0,6.0]|
|     243.0| 84|         7|     21|[243.0,21.0,7.0]|
|     243.0| 72|         8|     21|[243.0,21.0

In [17]:
testfinal = output1.select('NEW','TOT')

In [18]:
df1.select('EncodedUSN').show(50)

+----------+
|EncodedUSN|
+----------+
|     242.0|
|     242.0|
|     242.0|
|     242.0|
|     242.0|
|     242.0|
|     242.0|
|     242.0|
|     243.0|
|     243.0|
|     243.0|
|     243.0|
|     243.0|
|     243.0|
|     243.0|
|     243.0|
|     244.0|
|     244.0|
|     244.0|
|     244.0|
|     244.0|
|     244.0|
|     244.0|
|     244.0|
|     147.0|
|     147.0|
|     147.0|
|     147.0|
|     147.0|
|     147.0|
|     147.0|
|     147.0|
|     178.0|
|     178.0|
|     178.0|
|     178.0|
|     178.0|
|     178.0|
|     178.0|
|     178.0|
|     245.0|
|     245.0|
|     245.0|
|     245.0|
|     245.0|
|     245.0|
|     245.0|
|     245.0|
|     246.0|
|     246.0|
+----------+
only showing top 50 rows



In [19]:
StringIndexer(inputCols=["USN"], outputCols=["newUSN1"])
encoder = OneHotEncoder(
    inputCols=["newUSN1"],  
    outputCols=["newUSN"]
)

In [20]:
indexer = StringIndexer(inputCol="USN", outputCol="newUSN").fit(dffinal)
indexed_df = indexer.transform(dffinal)
indexed_df.show()

+----------+---+----------+-------+------+
|       USN|TOT|SCODEFINAL|NEWDATE|newUSN|
+----------+---+----------+-------+------+
|4JN18CS001| 73|         1|     19| 242.0|
|4JN18CS001| 76|         2|     19| 242.0|
|4JN18CS001| 68|         3|     19| 242.0|
|4JN18CS001| 57|         4|     19| 242.0|
|4JN18CS001| 87|         5|     19| 242.0|
|4JN18CS001| 72|         6|     19| 242.0|
|4JN18CS001| 86|         7|     19| 242.0|
|4JN18CS001| 59|         8|     19| 242.0|
|4JN18CS002| 84|         1|     19| 243.0|
|4JN18CS002| 81|         2|     19| 243.0|
|4JN18CS002| 68|         3|     19| 243.0|
|4JN18CS002| 63|         4|     19| 243.0|
|4JN18CS002| 89|         5|     19| 243.0|
|4JN18CS002| 75|         6|     19| 243.0|
|4JN18CS002| 84|         7|     19| 243.0|
|4JN18CS002| 72|         8|     19| 243.0|
|4JN18CS003| 81|         1|     19| 244.0|
|4JN18CS003| 73|         2|     19| 244.0|
|4JN18CS003| 81|         3|     19| 244.0|
|4JN18CS003| 72|         4|     19| 244.0|
+----------

### Prediction

In [21]:
predictions = regressor.transform(testfinal)
predictions.select(['NEW','TOT','prediction']).show(25)

+----------------+---+------------------+
|             NEW|TOT|        prediction|
+----------------+---+------------------+
|[242.0,21.0,1.0]| 73| 46.52975916570128|
|[242.0,21.0,2.0]| 76|49.036961065048345|
|[242.0,21.0,3.0]| 68|  51.5441629643954|
|[242.0,21.0,4.0]| 57|54.051364863742464|
|[242.0,21.0,5.0]| 87|56.558566763089516|
|[242.0,21.0,6.0]| 72| 59.06576866243657|
|[242.0,21.0,7.0]| 86|61.572970561783634|
|[242.0,21.0,8.0]| 59| 64.08017246113069|
|[243.0,21.0,1.0]| 84|   46.575806046176|
|[243.0,21.0,2.0]| 81|49.083007945523065|
|[243.0,21.0,3.0]| 68| 51.59020984487012|
|[243.0,21.0,4.0]| 63| 54.09741174421718|
|[243.0,21.0,5.0]| 89|56.604613643564235|
|[243.0,21.0,6.0]| 75| 59.11181554291129|
|[243.0,21.0,7.0]| 84|61.619017442258354|
|[243.0,21.0,8.0]| 72|  64.1262193416054|
|[244.0,21.0,1.0]| 81|  46.6218529266507|
|[244.0,21.0,2.0]| 73| 49.12905482599777|
|[244.0,21.0,3.0]| 81| 51.63625672534482|
|[244.0,21.0,4.0]| 72| 54.14345862469189|
|[244.0,21.0,5.0]| 57| 56.65066052